In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
;; Piece classification: determines whether path matters
(defun sliding-piece-p (ptype)
  "Sliding pieces (Q/R/B) attack along lines - path can be blocked"
  (declare (xargs :guard t))
  (member-equal ptype '(:queen :rook :bishop)))


Since SLIDING-PIECE-P is non-recursive, its admission is trivial. 
We observe that the type of SLIDING-PIECE-P is described by the theorem
(OR (CONSP (SLIDING-PIECE-P PTYPE)) (EQUAL (SLIDING-PIECE-P PTYPE) NIL)).
We used the :type-prescription rule MEMBER-EQUAL.

Computing the guard conjecture for SLIDING-PIECE-P....

The guard conjecture for SLIDING-PIECE-P is trivial to prove, given
the :executable-counterpart of TRUE-LISTP.  SLIDING-PIECE-P is compliant
with Common Lisp.

Summary
Form:  ( DEFUN SLIDING-PIECE-P ...)
Rules: ((:EXECUTABLE-COUNTERPART TRUE-LISTP)
        (:TYPE-PRESCRIPTION MEMBER-EQUAL))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 SLIDING-PIECE-P


In [3]:
(defun jumping-piece-p (ptype)
  "Jumping pieces (N) and contact pieces (P/K) - no blockable path"
  (declare (xargs :guard t))
  (member-equal ptype '(:knight :pawn :king)))


Since JUMPING-PIECE-P is non-recursive, its admission is trivial. 
We observe that the type of JUMPING-PIECE-P is described by the theorem
(OR (CONSP (JUMPING-PIECE-P PTYPE)) (EQUAL (JUMPING-PIECE-P PTYPE) NIL)).
We used the :type-prescription rule MEMBER-EQUAL.

Computing the guard conjecture for JUMPING-PIECE-P....

The guard conjecture for JUMPING-PIECE-P is trivial to prove, given
the :executable-counterpart of TRUE-LISTP.  JUMPING-PIECE-P is compliant
with Common Lisp.

Summary
Form:  ( DEFUN JUMPING-PIECE-P ...)
Rules: ((:EXECUTABLE-COUNTERPART TRUE-LISTP)
        (:TYPE-PRESCRIPTION MEMBER-EQUAL))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 JUMPING-PIECE-P


In [4]:
(defun piece-type-p (ptype)
  (declare (xargs :guard t))
  (or (sliding-piece-p ptype)
      (jumping-piece-p ptype)))


Since PIECE-TYPE-P is non-recursive, its admission is trivial.  We
observe that the type of PIECE-TYPE-P is described by the theorem 
(OR (CONSP (PIECE-TYPE-P PTYPE)) (EQUAL (PIECE-TYPE-P PTYPE) NIL)).
We used the :type-prescription rules JUMPING-PIECE-P and SLIDING-PIECE-P.

Computing the guard conjecture for PIECE-TYPE-P....

The guard conjecture for PIECE-TYPE-P is trivial to prove.  PIECE-TYPE-P
is compliant with Common Lisp.

Summary
Form:  ( DEFUN PIECE-TYPE-P ...)
Rules: ((:TYPE-PRESCRIPTION JUMPING-PIECE-P)
        (:TYPE-PRESCRIPTION SLIDING-PIECE-P))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 PIECE-TYPE-P


In [5]:
(encapsulate
  ;; Signatures for abstract attack predicates
  (((attacker-can-reach * * *) => *)      ; attacker-type, attacker-pos, target-pos
   ((path-clear-p * * *) => *)            ; attacker-pos, target-pos, board-state
   ((attack-valid-p * * * * *) => *))     ; type, attacker-pos, target-pos, board, is-slider
  
  ;; Witness functions (trivial - just need consistency)
  (local (defun attacker-can-reach (atype apos tpos)
           (declare (ignore atype apos tpos)) t))
  (local (defun path-clear-p (apos tpos board)
           (declare (ignore apos tpos board)) t))
  (local (defun attack-valid-p (atype apos tpos board is-slider)
           (declare (ignore atype apos tpos board is-slider)) t))
  
  ;; KEY CONSTRAINT: Attack validity depends on exactly these components
  ;; For sliding pieces: need reach AND clear path
  ;; For jumping pieces: need reach only (no path to block)
  
  (defthm attack-components
    (implies is-slider
             (equal (attack-valid-p atype apos tpos board is-slider)
                    (and (attacker-can-reach atype apos tpos)
                         (path-clear-p apos tpos board)))))
  
  (defthm attack-components-non-slider
    (implies (not is-slider)
             (equal (attack-valid-p atype apos tpos board is-slider)
                    (attacker-can-reach atype apos tpos)))))


To verify that the five encapsulated events correctly extend the current
theory we will evaluate them.  The theory thus constructed is only
ephemeral.

Encapsulated Events:


ACL2 !>>(LOCAL (DEFUN ATTACKER-CAN-REACH (ATYPE APOS TPOS)
                 (DECLARE (IGNORE ATYPE APOS TPOS))
                 T))

Since ATTACKER-CAN-REACH is non-recursive, its admission is trivial.
We observe that the type of ATTACKER-CAN-REACH is described by the
theorem (EQUAL (ATTACKER-CAN-REACH ATYPE APOS TPOS) T).  

Summary
Form:  ( DEFUN ATTACKER-CAN-REACH ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
ATTACKER-CAN-REACH


ACL2 !>>(LOCAL (DEFUN PATH-CLEAR-P (APOS TPOS BOARD)
                 (DECLARE (IGNORE APOS TPOS BOARD))
                 T))

Since PATH-CLEAR-P is non-recursive, its admission is trivial.  We
observe that the type of PATH-CLEAR-P is described by the theorem 
(EQUAL (PATH-CLEAR-P APOS TPOS BOARD) T).  

Summary
Form:  ( DEFUN PATH-CLEAR-P ...)
Rules: NIL

In [6]:
;; Move effects (abstract)
(encapsulate
  (((move-changes-king-pos * *) => *)     ; move, old-king-pos -> bool
   ((move-captures-attacker * *) => *)    ; move, attacker-pos -> bool  
   ((move-blocks-path * * * *) => *))     ; move, apos, tpos, board -> bool
  
  (local (defun move-changes-king-pos (move old-kpos)
           (declare (ignore move old-kpos)) nil))
  (local (defun move-captures-attacker (move apos)
           (declare (ignore move apos)) nil))
  (local (defun move-blocks-path (move apos tpos board)
           (declare (ignore move apos tpos board)) nil))
  
  ;; No constraints needed - these just classify what a move does
  )


To verify that the three encapsulated events correctly extend the current
theory we will evaluate them.  The theory thus constructed is only
ephemeral.

Encapsulated Events:


ACL2 !>>(LOCAL (DEFUN MOVE-CHANGES-KING-POS (MOVE OLD-KPOS)
                 (DECLARE (IGNORE MOVE OLD-KPOS))
                 NIL))

Since MOVE-CHANGES-KING-POS is non-recursive, its admission is trivial.
We observe that the type of MOVE-CHANGES-KING-POS is described by the
theorem (EQUAL (MOVE-CHANGES-KING-POS MOVE OLD-KPOS) NIL).  

Summary
Form:  ( DEFUN MOVE-CHANGES-KING-POS ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
MOVE-CHANGES-KING-POS


ACL2 !>>(LOCAL (DEFUN MOVE-CAPTURES-ATTACKER (MOVE APOS)
                 (DECLARE (IGNORE MOVE APOS))
                 NIL))

Since MOVE-CAPTURES-ATTACKER is non-recursive, its admission is trivial.
We observe that the type of MOVE-CAPTURES-ATTACKER is described by
the theorem (EQUAL (MOVE-CAPTURES-ATTACKER MOVE APOS) NIL).  

Summary
Fo

In [7]:
(defun escape-type-p (etype)
  (declare (xargs :guard t))
  (member-equal etype '(:king-move :capture :block)))


Since ESCAPE-TYPE-P is non-recursive, its admission is trivial.  We
observe that the type of ESCAPE-TYPE-P is described by the theorem
(OR (CONSP (ESCAPE-TYPE-P ETYPE)) (EQUAL (ESCAPE-TYPE-P ETYPE) NIL)).
We used the :type-prescription rule MEMBER-EQUAL.

Computing the guard conjecture for ESCAPE-TYPE-P....

The guard conjecture for ESCAPE-TYPE-P is trivial to prove, given the
:executable-counterpart of TRUE-LISTP.  ESCAPE-TYPE-P is compliant
with Common Lisp.

Summary
Form:  ( DEFUN ESCAPE-TYPE-P ...)
Rules: ((:EXECUTABLE-COUNTERPART TRUE-LISTP)
        (:TYPE-PRESCRIPTION MEMBER-EQUAL))
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 ESCAPE-TYPE-P


In [8]:
(defthm escaping-slider-attack-requires-component-change
  (implies (and is-slider
                ;; Attack was valid before
                (attack-valid-p atype apos tpos board-before is-slider)
                ;; Attack is invalid after
                (not (attack-valid-p atype apos tpos board-after is-slider)))
           ;; Then either reach failed or path blocked
           (or (not (attacker-can-reach atype apos tpos))
               (not (path-clear-p apos tpos board-after))))
  :hints (("Goal" :use attack-components)))


ACL2 Warning [Non-rec] in ( DEFTHM 
ESCAPING-SLIDER-ATTACK-REQUIRES-COMPONENT-CHANGE ...):  A :REWRITE
rule generated from ESCAPING-SLIDER-ATTACK-REQUIRES-COMPONENT-CHANGE
will be triggered only by terms containing the function symbol NOT,
which has a non-recursive definition.  Unless this definition is disabled,
this rule is unlikely ever to be used.


ACL2 Warning [Free] in ( DEFTHM 
ESCAPING-SLIDER-ATTACK-REQUIRES-COMPONENT-CHANGE ...):  A :REWRITE
rule generated from ESCAPING-SLIDER-ATTACK-REQUIRES-COMPONENT-CHANGE
contains the free variables IS-SLIDER and BOARD-BEFORE.  These variables
will be chosen by searching for instances of IS-SLIDER and 
(ATTACK-VALID-P ATYPE APOS TPOS BOARD-BEFORE IS-SLIDER) in the context
of the term being rewritten.  This is generally a severe restriction
on the applicability of a :REWRITE rule.  See :DOC free-variables.


ACL2 Warning [Use] in ( DEFTHM 
ESCAPING-SLIDER-ATTACK-REQUIRES-COMPONENT-CHANGE ...):  It is unusual
to :USE the formula of an enab

In [9]:
;; For non-sliders, only reach matters
(defthm escaping-jumper-attack-requires-reach-change  
  (implies (and (not is-slider)
                (attack-valid-p atype apos tpos board-before is-slider)
                (not (attack-valid-p atype apos tpos board-after is-slider)))
           (not (attacker-can-reach atype apos tpos)))
  :hints (("Goal" :use attack-components-non-slider)))


ACL2 Warning [Free] in ( DEFTHM ESCAPING-JUMPER-ATTACK-REQUIRES-REACH-CHANGE
...):  A :REWRITE rule generated from 
ESCAPING-JUMPER-ATTACK-REQUIRES-REACH-CHANGE contains the free variables
IS-SLIDER, BOARD-BEFORE and BOARD-AFTER.  These variables will be chosen
by searching for instances of (NOT IS-SLIDER), 
(ATTACK-VALID-P ATYPE APOS TPOS BOARD-BEFORE IS-SLIDER) and 
(NOT (ATTACK-VALID-P ATYPE APOS TPOS BOARD-AFTER IS-SLIDER)) in the
context of the term being rewritten.  This is generally a severe restriction
on the applicability of a :REWRITE rule.  See :DOC free-variables.


ACL2 Warning [Use] in ( DEFTHM ESCAPING-JUMPER-ATTACK-REQUIRES-REACH-CHANGE
...):  It is unusual to :USE the formula of an enabled :REWRITE or
:DEFINITION rule, so you may want to consider disabling 
(:REWRITE ATTACK-COMPONENTS-NON-SLIDER) in the hint provided for Goal.
See :DOC using-enabled-rules.

Goal'
Goal''

Q.E.D.

Summary
Form:  ( DEFTHM ESCAPING-JUMPER-ATTACK-REQUIRES-REACH-CHANGE ...)
Rules: ((:DEFINI

In [10]:
(encapsulate
  (((escape-classifies-as * * * * *) => *))  ; move, atype, apos, tpos, board
  
  ;; Witness function that returns classification based on hypothetical move effects
  ;; We use a simple witness: always returns :king-move (trivially satisfies all constraints
  ;; because constraints have hypotheses that won't all be satisfied simultaneously)
  (local (defun escape-classifies-as (move atype apos tpos board)
           (declare (ignore atype board))
           (cond ((move-changes-king-pos move tpos) :king-move)
                 ((move-captures-attacker move apos) :capture)
                 (t :block))))
  
  ;; CONSTRAINT: Every escape can be classified as one of the three types
  ;; This follows from the structure of attacks:
  ;; - If king moved: :king-move
  ;; - Else if attacker captured: :capture  
  ;; - Else (for sliders) path must be blocked: :block
  
  (defthm escape-classification-is-valid
    (escape-type-p (escape-classifies-as move atype apos tpos board)))
  
  ;; Classification matches what the move actually does
  (defthm king-move-classification
    (implies (move-changes-king-pos move old-kpos)
             (equal (escape-classifies-as move atype apos old-kpos board)
                    :king-move)))
  
  (defthm capture-classification  
    (implies (and (not (move-changes-king-pos move tpos))
                  (move-captures-attacker move apos))
             (equal (escape-classifies-as move atype apos tpos board)
                    :capture)))
  
  ;; For sliders: if not king-move and not capture, must be block
  (defthm block-classification
    (implies (and (not (move-changes-king-pos move tpos))
                  (not (move-captures-attacker move apos)))
             (equal (escape-classifies-as move atype apos tpos board)
                    :block))))


To verify that the five encapsulated events correctly extend the current
theory we will evaluate them.  The theory thus constructed is only
ephemeral.

Encapsulated Events:


ACL2 !>>(LOCAL (DEFUN ESCAPE-CLASSIFIES-AS (MOVE ATYPE APOS TPOS BOARD)
                 (DECLARE (IGNORE ATYPE BOARD))
                 (COND ((MOVE-CHANGES-KING-POS MOVE TPOS)
                        :KING-MOVE)
                       ((MOVE-CAPTURES-ATTACKER MOVE APOS)
                        :CAPTURE)
                       (T :BLOCK))))

Since ESCAPE-CLASSIFIES-AS is non-recursive, its admission is trivial.
We observe that the type of ESCAPE-CLASSIFIES-AS is described by the
theorem 
(AND (SYMBOLP (ESCAPE-CLASSIFIES-AS MOVE ATYPE APOS TPOS BOARD))
     (NOT (EQUAL (ESCAPE-CLASSIFIES-AS MOVE ATYPE APOS TPOS BOARD)
                 T))
     (NOT (EQUAL (ESCAPE-CLASSIFIES-AS MOVE ATYPE APOS TPOS BOARD)
                 NIL))).

Summary
Form:  ( DEFUN ESCAPE-CLASSIFIES-AS ...)
Rules: NIL
Time:  0.00 seconds (pro

In [11]:
(defthm three-ways-out-of-check
  (implies (and ;; Was in check (attack valid)
                (attack-valid-p atype apos tpos board-before is-slider)
                ;; Escaped check (attack now invalid)
                (not (attack-valid-p atype apos tpos board-after is-slider)))
           ;; Move classifies as one of the three escape types
           (escape-type-p (escape-classifies-as move atype apos tpos board-after)))
  :hints (("Goal" :use escape-classification-is-valid)))


ACL2 Warning [Non-rec] in ( DEFTHM THREE-WAYS-OUT-OF-CHECK ...):  A
:REWRITE rule generated from THREE-WAYS-OUT-OF-CHECK will be triggered
only by terms containing the function symbol ESCAPE-TYPE-P, which has
a non-recursive definition.  Unless this definition is disabled, this
rule is unlikely ever to be used.


ACL2 Warning [Free] in ( DEFTHM THREE-WAYS-OUT-OF-CHECK ...):  A :REWRITE
rule generated from THREE-WAYS-OUT-OF-CHECK contains the free variables
IS-SLIDER and BOARD-BEFORE.  These variables will be chosen by searching
for an instance of (ATTACK-VALID-P ATYPE APOS TPOS BOARD-BEFORE IS-SLIDER)
in the context of the term being rewritten.  This is generally a severe
restriction on the applicability of a :REWRITE rule.  See :DOC free-
variables.


ACL2 Warning [Subsume] in ( DEFTHM THREE-WAYS-OUT-OF-CHECK ...):  The
previously added rules ESCAPE-CLASSIFICATION-IS-VALID and ESCAPE-TYPE-P
subsume a newly proposed :REWRITE rule generated from 
THREE-WAYS-OUT-OF-CHECK, in the sense t

In [12]:
;; COROLLARY: Against jumping pieces, blocking is impossible
(defthm no-block-against-jumper
  (implies (and (not is-slider)  ; jumping piece
                (attack-valid-p atype apos tpos board-before is-slider)
                (not (attack-valid-p atype apos tpos board-after is-slider))
                (not (move-changes-king-pos move tpos))
                (not (move-captures-attacker move apos)))
           ;; This situation is impossible - must have moved or captured
           nil)
  :hints (("Goal" :use escaping-jumper-attack-requires-reach-change))
  :rule-classes nil)


ACL2 Warning [Use] in ( DEFTHM NO-BLOCK-AGAINST-JUMPER ...):  It is
unusual to :USE the formula of an enabled :REWRITE or :DEFINITION rule,
so you may want to consider disabling 
(:REWRITE ESCAPING-JUMPER-ATTACK-REQUIRES-REACH-CHANGE) in the hint
provided for Goal.  See :DOC using-enabled-rules.

Goal'
Goal''

Q.E.D.

Summary
Form:  ( DEFTHM NO-BLOCK-AGAINST-JUMPER ...)
Rules: ((:CONGRUENCE IFF-IMPLIES-EQUAL-NOT)
        (:DEFINITION NOT)
        (:EXECUTABLE-COUNTERPART NOT)
        (:REWRITE ATTACK-COMPONENTS-NON-SLIDER))
Hint-events: ((:USE ESCAPING-JUMPER-ATTACK-REQUIRES-REACH-CHANGE))
Warnings:  Use
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
Prover steps counted:  254
 NO-BLOCK-AGAINST-JUMPER
